In [1]:
import sys
from os import system
import os
import pandas as pd
import numpy as np
import time
import copy
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
from sklearn.metrics import accuracy_score
from sklearn.externals.joblib import dump, load

# sys.path.insert(1, os.path.join(sys.path[0], 'datagen/img_generator/'))
# from network import Net, Net_Regressor

In [2]:
data = pd.read_csv('classifier_files/data.csv')
print "Total length of data: ", data.shape
data[0:10]

Total length of data:  (8619, 19)


,true_init_x,true_init_y,true_init_z,jitter_coef,var_sum,diff_x,diff_y,diff_z,diff_phi,diff_theta,diff_psi,r_std,phi_std,theta_std,psi_std,Tf,MP_Method,Cost,Status
0,0.104533,-0.018855,-2.245773,2.780655,0.212268,0.057597,-3.303025,0.143396,0.055103,0.049895,0.814931,0.172320,0.003544,0.006260,0.030144,2.561507,min_vel,666.401116,SUCCESS
1,0.118917,-0.371141,-2.239867,10.727064,0.720797,0.079548,-1.481881,0.141947,0.055103,0.049895,-0.414747,0.694450,0.011473,0.001152,0.013722,1.209763,min_vel,139.277568,SUCCESS
2,0.125599,-0.698408,-2.228797,10.727064,0.033839,-0.633924,-0.945962,1.452816,0.055103,0.049895,0.470498,0.000000,0.005313,0.001731,0.026794,1.474120,min_vel,244.203045,SUCCESS
3,0.087731,-1.025364,-2.148764,10.727064,0.171747,-0.367246,-1.142579,-0.304712,0.055103,0.049895,0.299181,0.138481,0.006888,0.002289,0.024089,1.021753,min_vel,109.635576,SUCCESS
4,-0.013654,-1.178420,-2.078171,10.727064,0.459846,-0.252224,-1.408724,-0.318564,0.055103,0.049895,-0.107583,0.431646,0.007526,0.001927,0.018747,1.191425,min_vel,153.026475,SUCCESS
5,-0.105886,-1.429338,-2.044954,10.727064,0.024708,-0.043336,-0.706722,0.232820,0.055103,0.049895,0.080075,0.000000,0.008324,0.012506,0.003878,0.654845,min_vel,44.141688,SUCCESS
6,-0.135498,-1.594819,-2.034136,10.727064,0.257183,-0.040947,-1.477477,0.548405,0.055103,0.049895,-0.226680,0.219388,0.007363,0.010171,0.020261,1.273571,min_vel,169.766987,SUCCESS
7,-0.160248,-1.900392,-1.990043,1.880817,1.195650,0.308336,-2.382304,-0.196460,0.055103,0.049895,-0.059965,1.136596,0.008237,0.000000,0.050817,1.894183,min_vel,442.897805,SUCCESS
8,-0.134419,-2.413613,-1.939049,1.880817,1.102345,0.182406,-2.400703,0.023451,0.055103,0.049895,-0.061924,1.046051,0.005625,0.000000,0.050669,1.892353,min_vel,421.933112,SUCCESS
9,-0.064802,-2.988882,-1.924189,1.880817,0.251579,0.266086,-1.512065,-0.201828,0.055103,0.049895,-0.381524,0.242358,0.002321,0.000000,0.006900,1.252730,min_vel,155.128768,SUCCESS


### Classify the data points into lists considering the initial positions

In [3]:
eps = 0.1 # if the difference is as much as eps, then classify them into the same list
N_list = 1500 # number of lists
data_list = {}
data_list_no_failures = {} 
for i in range(N_list):
    list_name = "list_" + str(i+1)
    data_list[list_name] = []
    data_list_no_failures[list_name] = []
    
data_arr = data.values
N_array = data_arr.shape[0]
duplicate_entries = 0
for i in range(N_array):
    element = data_arr[i,:]
    x1,y1,z1,cost1 = element[0], element[1], element[2], element[17] 
    
    appending_complete = False
    for j in range(N_list):
        
        if appending_complete:
            break
        
        list_name = "list_" + str(j+1)
        if data_list[list_name]:
            for k in range(len(data_list[list_name])):
                element_compare = data_list[list_name][k]
                x2,y2,z2,cost2 = element_compare[0], element_compare[1], element_compare[2], element_compare[17]
                
                # if the initial positions of data points are in eps range, then classify them into the same list
                if (x2 - eps <= x1 <= x2 + eps) and (y2 - eps <= y1 <= y2 + eps)  and (z2 - eps <= z1 <= z2 + eps):
                    # if the cost values of different data points are exactly same, this means a duplicate data entry
                    # in this case, do not add the data into the lists
                    appending_complete = True
                    if cost2 - eps <= cost1 <= cost2 + eps:
                        duplicate_entries += 1                        
                        break
                    data_list[list_name].append(list(element[0:-1]))
                    
                    if cost1 < 1e12:
                        data_list_no_failures[list_name].append(list(element[0:-1]))
                    break
        else:
            # if this data point is not near of any other data points, put it into a new list
            data_list[list_name].append(list(element[0:-1]))
#             print "No data in " + list_name + " x1-y1-z1 is added here." 
            appending_complete = True
    
print "Process is completed."
print "Number of duplicate entries: ", duplicate_entries

Process is completed.
Number of duplicate entries:  202


### How many data points are there in each list?

In [4]:
N_sum = 0
N_sum_no_failures = 0
for k in range(N_list):
    list_name = "list_" + str(k+1)
    N_sum += len(data_list[list_name])
    N_sum_no_failures += len(data_list_no_failures[list_name])
    
print "Total number of same entries: ", duplicate_entries
print "Total number of data points: ", len(data.values)
print "Total number of classified data points: ", N_sum
print "Total number of data points with no failures: ", N_sum_no_failures

Total number of same entries:  202
Total number of data points:  8619
Total number of classified data points:  8417
Total number of data points with no failures:  6789


### Sort the data points by their cost value

In [5]:
from operator import itemgetter
for k in range(N_list):
    list_name = "list_" + str(k+1)
    data_list[list_name] = sorted(data_list[list_name], key=itemgetter(17))
    data_list_no_failures[list_name] = sorted(data_list_no_failures[list_name], key=itemgetter(17))

### Let us take a glance at the lists, we filled up

In [9]:
for k in range(N_list):
    list_name = "list_" + str(k+1)
    if data_list_no_failures[list_name]:
        first = data_list_no_failures[list_name][0]
        last = data_list_no_failures[list_name][-1]
        print "\nlist_"+str(k+1) + " - Length: " + str(len(data_list_no_failures[list_name]))
        print "Method: {6}, jitter: {5:.4}, var_sum: {4:.4}, x: {0:.4}, y: {1:.4}, z: {2:.4}, cost: {3:.6} ".format(first[0], first[1], first[2], first[17], first[4], first[3], first[16])
        print "Method: {6}, jitter: {5:.4}, var_sum: {4:.4}, x: {0:.4}, y: {1:.4}, z: {2:.4}, cost: {3:.6} ".format(last[0], last[1], last[2], last[17], last[4], last[3], last[16])


list_1 - Length: 1278
Method: min_jerk_full_stop, jitter: 9.008, var_sum: 0.05589, x: -0.2227, y: -0.5235, z: -1.802, cost: 1.92094 
Method: min_vel, jitter: 11.01, var_sum: 0.05026, x: -0.04894, y: -0.7469, z: -1.964, cost: 15074.7 

list_2 - Length: 11
Method: min_vel, jitter: 9.097, var_sum: 0.839, x: 0.04469, y: -0.3854, z: -2.195, cost: 114.426 
Method: min_vel, jitter: 10.61, var_sum: 0.06262, x: -0.04607, y: -0.2522, z: -2.31, cost: 2984.6 

list_3 - Length: 146
Method: min_jerk_full_stop, jitter: 9.069, var_sum: 0.01805, x: 0.03125, y: -1.142, z: -2.148, cost: 7.89772 
Method: min_vel, jitter: 8.132, var_sum: 0.08938, x: -0.109, y: -0.8297, z: -2.079, cost: 12118.9 

list_4 - Length: 68
Method: min_acc, jitter: 9.862, var_sum: 0.02737, x: 0.03978, y: -1.1, z: -2.179, cost: 75.5992 
Method: min_jerk, jitter: 9.282, var_sum: 0.07665, x: 0.1551, y: -1.092, z: -1.879, cost: 16036.2 

list_5 - Length: 148
Method: min_jerk, jitter: 9.084, var_sum: 0.1133, x: -0.1251, y: -1.572, z: -

### We need to convert algorithm types into numbers

In [10]:
algorithm_dict = {"min_vel":0, "min_acc":1, "min_jerk":2, "min_jerk_full_stop":3}
algorithms = ["min_vel", "min_acc", "min_jerk", "min_jerk_full_stop"]

for k in range(N_list):
    list_name = "list_" + str(k+1)
    if data_list[list_name]:
        for j in range(len(data_list[list_name])):
            index = data_list[list_name][j][16]
            data_list[list_name][j][16] = int(algorithm_dict[index])
            
    if data_list_no_failures[list_name]:
        for j in range(len(data_list_no_failures[list_name])):
            index = data_list_no_failures[list_name][j][16]
            data_list_no_failures[list_name][j][16] = int(algorithm_dict[index])

### Details of cost values in the lists

In [11]:
cost_median = {}
for k in range(N_list):
    list_name = "list_" + str(k+1)
    if data_list[list_name]:
        arr = np.array(data_list[list_name],dtype=np.float)
        print "\nList: ", list_name
        print "Cost values: ", arr[:,17]
        print "Mean cost: ", np.mean(arr[:,17])
        print "Median cost: ", np.median(arr[:,17])
        print "Std cost: ", np.std(arr[:,17])
        
        cost_median[list_name] = np.median(arr[:,17])


List:  list_1
Cost values:  [1.92094187e+00 3.00960915e+00 5.50329599e+00 ... 1.37767905e+04
 1.40785814e+04 1.50747106e+04]
Mean cost:  980.8173503041813
Median cost:  623.6013043976685
Std cost:  1475.2577692611267

List:  list_2
Cost values:  [ 114.42550345  139.27756753  155.05152425  223.50251003  319.81671848
  461.07721816  464.39562259  832.96011409 1190.53452004 1268.65201592
 1852.65022005 2984.59591332]
Mean cost:  833.9116206597608
Median cost:  462.73642037171396
Std cost:  834.3120647179087

List:  list_3
Cost values:  [7.89771980e+00 1.11425676e+01 1.82802043e+01 1.89704070e+01
 1.93600807e+01 1.94557359e+01 2.64587145e+01 2.80021374e+01
 2.86595333e+01 4.50001567e+01 5.76386773e+01 7.16729763e+01
 7.25809691e+01 8.07494373e+01 8.11546257e+01 8.40370534e+01
 9.53471438e+01 9.63490596e+01 1.00026452e+02 1.16611713e+02
 1.17364861e+02 1.23142671e+02 1.27445938e+02 1.29667191e+02
 1.34061013e+02 1.34681472e+02 1.41688236e+02 1.42835653e+02
 1.46942139e+02 1.50786174e+02 1.

Median cost:  214.31604997326573
Std cost:  209.244027235991

List:  list_147
Cost values:  [ 385.99469452  397.40499604  424.98312043  509.66681346  534.63503979
  705.27721817  879.86967777  949.9061744  1000.68024389 1170.55260944
 1441.69256472 5220.44925651]
Mean cost:  1135.0927007624653
Median cost:  792.5734479699152
Std cost:  1273.0846233919422

List:  list_148
Cost values:  [111.82072685 282.53390034 435.28617616 597.02002823]
Mean cost:  356.665207892921
Median cost:  358.91003824784866
Std cost:  179.85825458534592

List:  list_149
Cost values:  [ 36.91212123 395.893178  ]
Mean cost:  216.40264961658448
Median cost:  216.40264961658448
Std cost:  179.490528381718

List:  list_150
Cost values:  [ 53.13428064 281.38611668]
Mean cost:  167.2601986606897
Median cost:  167.2601986606897
Std cost:  114.12591801781412

List:  list_151
Cost values:  [ 31.23367272 103.67978906 142.30526639 188.44030874]
Mean cost:  116.41475922738543
Median cost:  122.9925277224616
Std cost:  57.61

Mean cost:  147.78863482964857
Median cost:  147.78863482964857
Std cost:  19.7951992006639

List:  list_326
Cost values:  [124.46427595]
Mean cost:  124.4642759546052
Median cost:  124.4642759546052
Std cost:  0.0

List:  list_327
Cost values:  [1.e+12]
Mean cost:  1000000000000.0
Median cost:  1000000000000.0
Std cost:  0.0

List:  list_328
Cost values:  [107.79231245 278.67329843 280.42042816 331.59312871 344.46101536]
Mean cost:  268.5880366212722
Median cost:  280.420428156594
Std cost:  84.64435413779596

List:  list_329
Cost values:  [2.17789073e+02 1.00000000e+12]
Mean cost:  500000000108.89453
Median cost:  500000000108.89453
Std cost:  499999999891.10547

List:  list_330
Cost values:  [269.43897967 491.58084955]
Mean cost:  380.5099146126705
Median cost:  380.5099146126705
Std cost:  111.07093494008487

List:  list_331
Cost values:  [69.55973554]
Mean cost:  69.55973553585818
Median cost:  69.55973553585818
Std cost:  0.0

List:  list_332
Cost values:  [7.77732974e+02 1.00000

List:  list_538
Cost values:  [84.80061036]
Mean cost:  84.80061036381579
Median cost:  84.80061036381579
Std cost:  0.0

List:  list_539
Cost values:  [400.01029932 424.91361067 466.75755477]
Mean cost:  430.5604882525134
Median cost:  424.91361066816506
Std cost:  27.54044840376058

List:  list_540
Cost values:  [81.28706296]
Mean cost:  81.28706295622885
Median cost:  81.28706295622885
Std cost:  0.0

List:  list_541
Cost values:  [161.52966211]
Mean cost:  161.5296621131142
Median cost:  161.5296621131142
Std cost:  0.0

List:  list_542
Cost values:  [785.45806535]
Mean cost:  785.4580653538817
Median cost:  785.4580653538817
Std cost:  0.0

List:  list_543
Cost values:  [5.01110930e+02 8.44434498e+02 6.07328531e+03 1.00000000e+12]
Mean cost:  250000001854.70767
Median cost:  3458.8599060995616
Std cost:  433012700821.4033

List:  list_544
Cost values:  [813.34730416]
Mean cost:  813.3473041625119
Median cost:  813.3473041625119
Std cost:  0.0

List:  list_545
Cost values:  [1.e+12


List:  list_755
Cost values:  [12.97379723 36.51780178]
Mean cost:  24.74579950201698
Median cost:  24.74579950201698
Std cost:  11.772002275867223

List:  list_756
Cost values:  [36.06395688]
Mean cost:  36.063956879809496
Median cost:  36.063956879809496
Std cost:  0.0

List:  list_757
Cost values:  [955.73452971]
Mean cost:  955.7345297122715
Median cost:  955.7345297122715
Std cost:  0.0

List:  list_758
Cost values:  [34.48687792]
Mean cost:  34.48687791697744
Median cost:  34.48687791697744
Std cost:  0.0

List:  list_759
Cost values:  [32.91033037]
Mean cost:  32.91033036939517
Median cost:  32.91033036939517
Std cost:  0.0

List:  list_760
Cost values:  [119.61851428]
Mean cost:  119.61851427608185
Median cost:  119.61851427608185
Std cost:  0.0

List:  list_761
Cost values:  [63.28580862]
Mean cost:  63.28580861758552
Median cost:  63.28580861758552
Std cost:  0.0

List:  list_762
Cost values:  [63.28580862]
Mean cost:  63.28580861758552
Median cost:  63.28580861758552
Std co

Mean cost:  2568.608003719992
Median cost:  2568.608003719992
Std cost:  0.0

List:  list_952
Cost values:  [2568.60800372]
Mean cost:  2568.608003719992
Median cost:  2568.608003719992
Std cost:  0.0

List:  list_953
Cost values:  [2.35576014e+02 2.58754062e+02 1.00000000e+12]
Mean cost:  333333333498.11005
Median cost:  258.7540624816561
Std cost:  471404520674.51697

List:  list_954
Cost values:  [206.73291412 207.80888983]
Mean cost:  207.27090197236805
Median cost:  207.27090197236805
Std cost:  0.5379878560403171

List:  list_955
Cost values:  [479.68414498]
Mean cost:  479.6841449762071
Median cost:  479.6841449762071
Std cost:  0.0

List:  list_956
Cost values:  [235.42186629 672.28182976]
Mean cost:  453.851848024703
Median cost:  453.851848024703
Std cost:  218.42998173940413

List:  list_957
Cost values:  [ 95.75155568 209.46015255]
Mean cost:  152.60585411501177
Median cost:  152.60585411501177
Std cost:  56.85429843610577

List:  list_958
Cost values:  [4.69450010e+02 1.00

Std cost:  0.0

List:  list_1138
Cost values:  [601.86028266]
Mean cost:  601.8602826637433
Median cost:  601.8602826637433
Std cost:  0.0

List:  list_1139
Cost values:  [601.15050347]
Mean cost:  601.150503471765
Median cost:  601.150503471765
Std cost:  0.0

List:  list_1140
Cost values:  [146.47434712]
Mean cost:  146.47434712051228
Median cost:  146.47434712051228
Std cost:  0.0

List:  list_1141
Cost values:  [195.40534522]
Mean cost:  195.40534521592772
Median cost:  195.40534521592772
Std cost:  0.0

List:  list_1142
Cost values:  [172.15217842]
Mean cost:  172.1521784197351
Median cost:  172.1521784197351
Std cost:  0.0

List:  list_1143
Cost values:  [360.18674151]
Mean cost:  360.1867415086924
Median cost:  360.1867415086924
Std cost:  0.0

List:  list_1144
Cost values:  [262.55430105]
Mean cost:  262.5543010525223
Median cost:  262.5543010525223
Std cost:  0.0

List:  list_1145
Cost values:  [793.82829944]
Mean cost:  793.8282994373177
Median cost:  793.8282994373177
Std co

Std cost:  0.0

List:  list_1338
Cost values:  [4203.57799403]
Mean cost:  4203.577994032439
Median cost:  4203.577994032439
Std cost:  0.0

List:  list_1339
Cost values:  [784.39881312]
Mean cost:  784.3988131233658
Median cost:  784.3988131233658
Std cost:  0.0

List:  list_1340
Cost values:  [675.11199161]
Mean cost:  675.1119916108782
Median cost:  675.1119916108782
Std cost:  0.0

List:  list_1341
Cost values:  [26.82488298]
Mean cost:  26.824882977570766
Median cost:  26.824882977570766
Std cost:  0.0

List:  list_1342
Cost values:  [114.40566826]
Mean cost:  114.40566825890815
Median cost:  114.40566825890815
Std cost:  0.0

List:  list_1343
Cost values:  [10.34786641]
Mean cost:  10.347866412534357
Median cost:  10.347866412534357
Std cost:  0.0

List:  list_1344
Cost values:  [1.e+12]
Mean cost:  1000000000000.0
Median cost:  1000000000000.0
Std cost:  0.0

List:  list_1345
Cost values:  [504.9781007]
Mean cost:  504.9781006973568
Median cost:  504.9781006973568
Std cost:  0.0

In [13]:
print_lim = 100
for k in range(N_list):
    list_name = "list_" + str(k+1)
    algorithm_count = [0,0,0,0,0]
    
    if data_list_no_failures[list_name]:
        arr = np.array(data_list_no_failures[list_name],dtype=np.float)
        print "\nList: ", list_name
        print "Method - Cost"
        for i, (method, cost) in enumerate(zip(arr[:,16],arr[:,17])):
            if i < 10:
                print "Method: {0}, jitter: {1:.3}, cov_sum: {2:.5}, cost: {3:.5}".format(algorithms[int(method)], arr[i][3], arr[i][4], arr[i][17])
            
            if i < print_lim:
                algorithm_count[int(method)] += 1.
            elif i == print_lim:
                print "Limit Cost: ", cost
        
#         print "\nCost values: ", arr[:,17]
        print "\nMean cost: ", np.mean(arr[:,17])
        print "Median cost: ", np.median(arr[:,17])
        print "Std cost: ", np.std(arr[:,17])
        
        print "\nDistribution on the first {0} data points".format(print_lim)
        print "N_min_vel: {0:.3}".format(algorithm_count[0] / np.sum(algorithm_count))
        print "N_min_acc: {0:.3}".format(algorithm_count[1] / np.sum(algorithm_count))
        print "N_min_jerk: {0:.3}".format(algorithm_count[2] / np.sum(algorithm_count)) 
        print "N_min_jerk_stop: {0:.3}".format(algorithm_count[3] / np.sum(algorithm_count)) 


List:  list_1
Method - Cost
Method: min_jerk_full_stop, jitter: 9.01, cov_sum: 0.055891, cost: 1.9209
Method: min_jerk_full_stop, jitter: 9.49, cov_sum: 0.11975, cost: 3.0096
Method: min_acc, jitter: 8.05, cov_sum: 0.23245, cost: 5.5033
Method: min_jerk_full_stop, jitter: 9.04, cov_sum: 0.026759, cost: 7.281
Method: min_jerk_full_stop, jitter: 10.1, cov_sum: 0.50607, cost: 9.6375
Method: min_jerk_full_stop, jitter: 9.17, cov_sum: 0.032134, cost: 11.328
Method: min_jerk_full_stop, jitter: 9.35, cov_sum: 0.71526, cost: 15.571
Method: min_acc, jitter: 8.81, cov_sum: 0.4881, cost: 16.055
Method: min_jerk, jitter: 9.55, cov_sum: 0.020505, cost: 16.579
Method: min_jerk_full_stop, jitter: 7.89, cov_sum: 0.045387, cost: 19.587
Limit Cost:  73.16624889469514

Mean cost:  981.0633723970006
Median cost:  622.7685590339452
Std cost:  1475.8086034229937

Distribution on the first 100 data points
N_min_vel: 0.06
N_min_acc: 0.28
N_min_jerk: 0.25
N_min_jerk_stop: 0.41

List:  list_2
Method - Cost
Met

Median cost:  452.1648564594359
Std cost:  156.95415240427806

Distribution on the first 100 data points
N_min_vel: 0.0
N_min_acc: 0.0
N_min_jerk: 1.0
N_min_jerk_stop: 0.0

List:  list_136
Method - Cost
Method: min_jerk, jitter: 8.11, cov_sum: 0.017561, cost: 60.108
Method: min_jerk, jitter: 7.2, cov_sum: 0.02177, cost: 74.339
Method: min_vel, jitter: 10.3, cov_sum: 0.022633, cost: 96.475
Method: min_jerk, jitter: 1.52, cov_sum: 0.40243, cost: 193.49
Method: min_vel, jitter: 8.09, cov_sum: 0.027934, cost: 195.19
Method: min_jerk, jitter: 2.16, cov_sum: 0.22542, cost: 222.93
Method: min_jerk, jitter: 9.63, cov_sum: 0.77422, cost: 576.72
Method: min_jerk, jitter: 8.54, cov_sum: 0.42375, cost: 644.62
Method: min_jerk, jitter: 1.96, cov_sum: 0.65025, cost: 673.28
Method: min_jerk, jitter: 1.43, cov_sum: 0.63197, cost: 907.2

Mean cost:  364.4357482755148
Median cost:  209.0606331679819
Std cost:  289.82958372111943

Distribution on the first 100 data points
N_min_vel: 0.2
N_min_acc: 0.0
N_

Method: min_acc, jitter: 8.91, cov_sum: 0.81886, cost: 273.53
Method: min_jerk_full_stop, jitter: 1.79, cov_sum: 0.36705, cost: 604.22
Method: min_jerk_full_stop, jitter: 2.25, cov_sum: 0.4674, cost: 624.75
Method: min_jerk_full_stop, jitter: 2.07, cov_sum: 0.26236, cost: 1302.6

Mean cost:  673.7496235808277
Median cost:  203.86606045563718
Std cost:  1117.6924353846766

Distribution on the first 100 data points
N_min_vel: 0.0909
N_min_acc: 0.364
N_min_jerk: 0.0
N_min_jerk_stop: 0.545

List:  list_295
Method - Cost
Method: min_jerk_full_stop, jitter: 8.66, cov_sum: 0.20967, cost: 16.264
Method: min_acc, jitter: 10.3, cov_sum: 0.090178, cost: 31.709
Method: min_jerk_full_stop, jitter: 8.91, cov_sum: 0.89036, cost: 47.071
Method: min_acc, jitter: 8.85, cov_sum: 0.44418, cost: 52.135
Method: min_acc, jitter: 0.913, cov_sum: 0.98614, cost: 177.8
Method: min_vel, jitter: 1.22, cov_sum: 0.65696, cost: 178.96
Method: min_jerk_full_stop, jitter: 8.33, cov_sum: 0.34437, cost: 204.6
Method: min

Method: min_jerk, jitter: 2.0, cov_sum: 1.2549, cost: 480.94

Mean cost:  351.35318806610246
Median cost:  351.35318806610246
Std cost:  129.58268403041498

Distribution on the first 100 data points
N_min_vel: 0.0
N_min_acc: 0.0
N_min_jerk: 1.0
N_min_jerk_stop: 0.0

List:  list_709
Method - Cost
Method: min_jerk_full_stop, jitter: 9.62, cov_sum: 0.51486, cost: 287.56
Method: min_acc, jitter: 8.61, cov_sum: 0.061729, cost: 296.07

Mean cost:  291.8185474519113
Median cost:  291.8185474519113
Std cost:  4.255038962727156

Distribution on the first 100 data points
N_min_vel: 0.0
N_min_acc: 0.5
N_min_jerk: 0.0
N_min_jerk_stop: 0.5

List:  list_713
Method - Cost
Method: min_jerk_full_stop, jitter: 1.8, cov_sum: 0.23568, cost: 125.46

Mean cost:  125.45932415688594
Median cost:  125.45932415688594
Std cost:  0.0

Distribution on the first 100 data points
N_min_vel: 0.0
N_min_acc: 0.0
N_min_jerk: 0.0
N_min_jerk_stop: 1.0

List:  list_716
Method - Cost
Method: min_jerk_full_stop, jitter: 8.63,

### Dataset Preparation

In [50]:
cost_upper_lim = 1e9

method_list = ["min_vel", "min_acc", "min_jerk", "min_jerk_full_stop"]

mp_classifier_data = []
time_regressor_data = []

for k in range(N_list):
    list_name = "list_" + str(k+1)
    time_data_appending = False
    if data_list[list_name]:
        print "\nList: ", list_name
        arr = np.array(data_list[list_name],dtype=np.float32).reshape(-1, 18)
        best_method = int(arr[0][16])
        for i in range(len(data_list[list_name])):
            cost_val = arr[i][17]
            # check if cost of each data point is less than mean cost and upper limit of cost value
            if cost_val <= cost_median[list_name] and cost_val < cost_upper_lim: 
                
#                 print "Data point, dx: {0:.4}, dy: {1:.4}, dz: {2:.4}, Tf: {3:.4}, v_avg: {4:.4}, cost:{5:.4}, method: {6}, type: {7}".format(arr[i][3],arr[i][4],arr[i][5],
#                                                                                                                                    arr[i][14],arr[i][15], arr[i][17], 
#                                                                                                                                    method_list[int(arr[i][16])],type_list[int(arr[i][13])])
                #if the output of current data point is consistent with the output of the data point, which has the lowest cost
                #then add it into the dataset (they should not conflict each other)
                if int(arr[i][16]) == best_method:
                    #true_init_x, true_init_y, true_init_z, blur_coeff, var_sum, diff_x, diff_y, diff_z, diff_phi, diff_theta, diff_psi, r_std, phi_std, theta_std, psi_std, Tf, MP_Method, Cost
                    mp_classifier_data.append([arr[i][5],arr[i][6],arr[i][7],arr[i][8],arr[i][9],arr[i][10], 
                                          arr[i][4],arr[i][11],arr[i][12],arr[i][13],arr[i][14],int(arr[i][16])])
                    print "MP Classifier, dx: {0:.4}, dy: {1:.4}, dz: {2:.4}, cost:{3:.4}, method: {4}".format(arr[i][5],arr[i][6],arr[i][7],
                                                                                                            arr[i][17], method_list[int(arr[i][16])])
                
                #If data point belongs time based algorithm
                #then add it into the time_regressor dataset
                if not time_data_appending: 
                    #true_init_x, true_init_y, true_init_z, blur_coeff, var_sum, diff_x, diff_y, diff_z, diff_phi, diff_theta, diff_psi, r_std, phi_std, theta_std, psi_std, Tf, MP_Method, Cost
                    time_regressor_data.append([arr[i][5],arr[i][6],arr[i][7],arr[i][8],arr[i][9],arr[i][10], 
                                          arr[i][4],arr[i][11],arr[i][12],arr[i][13],arr[i][14],arr[i][15]])
                    
                    time_data_appending = True
                    print "Time Regressor, dx: {0:.4}, dy: {1:.4}, dz: {2:.4}, cost:{3:.4}, Tf: {4:.4}".format(arr[i][5],arr[i][6],arr[i][7],
                                                                                                    arr[i][17], arr[i][15])
            elif cost_val > cost_upper_lim: #failed flights should be in dataset as well
                #true_init_x, true_init_y, true_init_z, blur_coeff, var_sum, diff_x, diff_y, diff_z, diff_phi, diff_theta, diff_psi, r_std, phi_std, theta_std, psi_std, Tf, MP_Method, Cost
                mp_classifier_data.append([arr[i][5],arr[i][6],arr[i][7],arr[i][8],arr[i][9],arr[i][10], 
                                          arr[i][4],arr[i][11],arr[i][12],arr[i][13],arr[i][14],4])
            
mp_classifier_data = np.array(mp_classifier_data)
time_regressor_data = np.array(time_regressor_data)

print ""
print "mp_classifier_data shape: ", mp_classifier_data.shape
print "time_regressor_data shape: ", time_regressor_data.shape


List:  list_1
MP Classifier, dx: 0.2524, dy: -3.577, dz: 0.2004, cost:542.2, method: min_vel
Time Regressor, dx: 0.2524, dy: -3.577, dz: 0.2004, cost:542.2, Tf: 2.53
MP Classifier, dx: 0.2341, dy: -3.524, dz: 0.4184, cost:597.0, method: min_vel
MP Classifier, dx: -0.06851, dy: -3.518, dz: -0.2731, cost:750.5, method: min_vel
MP Classifier, dx: 0.1984, dy: -3.487, dz: 0.4164, cost:951.3, method: min_vel
MP Classifier, dx: 0.05051, dy: -3.662, dz: -0.3812, cost:1.009e+03, method: min_vel
MP Classifier, dx: -0.1276, dy: -3.578, dz: -0.2199, cost:1.087e+03, method: min_vel

List:  list_2
MP Classifier, dx: 0.1513, dy: -2.767, dz: -1.057, cost:151.7, method: min_jerk_full_stop
Time Regressor, dx: 0.1513, dy: -2.767, dz: -1.057, cost:151.7, Tf: 1.904
MP Classifier, dx: 0.05416, dy: -2.987, dz: -0.5238, cost:165.6, method: min_jerk_full_stop
MP Classifier, dx: 0.3099, dy: -3.148, dz: -0.7137, cost:206.4, method: min_jerk_full_stop
MP Classifier, dx: 0.01995, dy: -2.943, dz: -0.9014, cost:210

List:  list_569
MP Classifier, dx: 0.3491, dy: -0.8031, dz: -0.8062, cost:680.4, method: min_jerk
Time Regressor, dx: 0.3491, dy: -0.8031, dz: -0.8062, cost:680.4, Tf: 0.8174

List:  list_570
MP Classifier, dx: 1.248, dy: -2.51, dz: -0.2395, cost:910.1, method: min_jerk
Time Regressor, dx: 1.248, dy: -2.51, dz: -0.2395, cost:910.1, Tf: 1.796

List:  list_571
MP Classifier, dx: 0.7625, dy: -1.606, dz: -0.8646, cost:925.5, method: min_jerk
Time Regressor, dx: 0.7625, dy: -1.606, dz: -0.8646, cost:925.5, Tf: 1.291

List:  list_572
MP Classifier, dx: 0.3871, dy: -1.268, dz: -0.1796, cost:1.063e+03, method: min_jerk
Time Regressor, dx: 0.3871, dy: -1.268, dz: -0.1796, cost:1.063e+03, Tf: 0.9061

List:  list_573

List:  list_574
MP Classifier, dx: -0.4016, dy: -1.025, dz: 0.8042, cost:1.835e+03, method: min_jerk
Time Regressor, dx: -0.4016, dy: -1.025, dz: 0.8042, cost:1.835e+03, Tf: 0.9218

List:  list_575

List:  list_576

List:  list_577
MP Classifier, dx: 0.07953, dy: -1.648, dz: 0.6861,

### Dataset Analysis

In [52]:
class_labels = {"min_vel":0, "min_acc":1, "min_jerk":2, "min_jerk_full_stop":3, "safe_mode":4}
N_length = float(mp_classifier_data.shape[0])
print "MP Dataset size: ",mp_classifier_data.shape 
print "N_min_vel: ",np.sum(mp_classifier_data[:,-1] == class_labels["min_vel"]) / N_length * 100
print "N_min_acc: ",np.sum(mp_classifier_data[:,-1] == class_labels["min_acc"]) / N_length * 100
print "N_min_jerk: ",np.sum(mp_classifier_data[:,-1] == class_labels["min_jerk"]) / N_length * 100
print "N_min_jerk_stop: ",np.sum(mp_classifier_data[:,-1] == class_labels["min_jerk_full_stop"]) / N_length * 100
print "N_safe_mode: ",np.sum(mp_classifier_data[:,-1] == class_labels["safe_mode"]) / N_length * 100

MP Dataset size:  (1327, 12)
N_min_vel:  10.399397136397889
N_min_acc:  15.749811605124343
N_min_jerk:  15.900527505651846
N_min_jerk_stop:  57.573474001507165
N_safe_mode:  0.37678975131876413


### Split of dataset as train, validation and test

In [60]:
MP_Test = 200
Time_Test = 75
np.random.shuffle(mp_classifier_data) # Train 1
np.random.shuffle(time_regressor_data) # Train 2

y1 = np.array(mp_classifier_data[:,-1],dtype=np.int)
y2 = np.array(time_regressor_data[:,-1],dtype=np.float)
X1 = mp_classifier_data[:,:-1]
X2 = time_regressor_data[:,:-1]

X1_test = X1[0:MP_Test,:]
y1_test = y1[0:MP_Test]
X2_test = X2[0:Time_Test,:]
y2_test = y2[0:Time_Test]

X1_train, X1_val, y1_train, y1_val = train_test_split(X1[MP_Test:,:], y1[MP_Test:], test_size=0.2, random_state=42)
X2_train, X2_val, y2_train, y2_val = train_test_split(X2[Time_Test:,:], y2[Time_Test:], test_size=0.2, random_state=42)
    
pickle.dump([X1_train, X1_val, X1_test, y1_train, y1_val, y1_test], open("classifier_files/dataset1.pkl","wb"), protocol=2)
pickle.dump([X2_train, X2_val, X2_test, y2_train, y2_val, y2_test], open("classifier_files/dataset2.pkl","wb"), protocol=2)

scaler1,scaler2 = StandardScaler(), StandardScaler()
scaler1.fit(X1_train)
scaler2.fit(X2_train)
X1_train, X2_train = scaler1.transform(X1_train), scaler2.transform(X2_train)
X1_val, X2_val = scaler1.transform(X1_val), scaler2.transform(X2_val)
X1_test, X2_test = scaler1.transform(X1_test), scaler2.transform(X2_test)

dump(scaler1, 'classifier_files/mp_scaler.bin', compress=True)
dump(scaler2, 'classifier_files/time_scaler.bin', compress=True)

print "X1_train: " + str(X1_train.shape) + " X2_train: " + str(X2_train.shape)
print "X1_val: " + str(X1_val.shape) + " X2_val: " + str(X2_val.shape) 
print "X1_test: " + str(X1_test.shape) + " X2_test: " + str(X2_test.shape) 

X1_train: (901, 11) X2_train: (310, 11)
X1_val: (226, 11) X2_val: (78, 11)
X1_test: (200, 11) X2_test: (75, 11)


### Train and predict functions

In [55]:
def predict(X, y, model, criterion, isClassifier):
    #Validation part
    model.eval()  # Set model to training mode
    
    inputs = torch.from_numpy(X).to(device)
    if isClassifier:
        labels = torch.from_numpy(y).to(device).long()
    else:
        labels = torch.from_numpy(y).to(device).float()

    outputs = model(inputs.float())
    if isClassifier:
        _, preds = torch.max(outputs, 1)

    loss = criterion(outputs, labels)
    
    if isClassifier:
        accuracy = torch.sum(preds == labels.data).item() / float(inputs.size(0))
        print "Test data, Loss: {0:.3}, Accuracy: {1:.4}".format(loss.item(), accuracy)
    else:
        print "Test data, Loss: {0:.3}\n".format(loss.item())
        print "Actual - Predicted"
        pred = outputs.detach().cpu().numpy()
        for i in range(y.shape[0]):
            print "{0:.3} - {1:.3}".format(y[i], outputs[i].item())
        
    

def shuffle_dataset(X, y):
    p = np.random.permutation(len(X))
    return X[p], y[p]

def train_model(isClassifier, X, y, X_val, y_val, model, criterion, optimizer, scheduler, minibatch_size, name, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_train_acc = 0.0
    best_acc = 0.0
    best_loss = 1e6
    best_train_loss = 1e6
    
    losses_train = []
    losses_val = []
    accuracy_train = []
    accuracy_val = []
    # path = F"/content/drive/My Drive/best_model.pt"
#     directory = path_name

#     if not os.path.exists(directory):
#         os.makedirs(directory)

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        X_train, y_train = shuffle_dataset(X, y)

        # Each epoch has a training and validation phase
        model.train()  # Set model to training mode
        running_loss = 0.0
        running_corrects = 0
        losses_iter = []
        accuracy_iter = []

        # Iterate over data.
        for i in range(0, X_train.shape[0], minibatch_size):
            # Get pair of (X, y) of the current minibatch/chunk             
            X_batch = X_train[i:i + minibatch_size]
            y_batch = y_train[i:i + minibatch_size]

            # get the inputs; data is a list of [inputs, labels]
            xbatch = torch.from_numpy(X_batch).to(device)
            
            if isClassifier:
                ybatch = torch.from_numpy(y_batch).to(device).long()
            else:
                ybatch = torch.from_numpy(y_batch).to(device).float()


            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(True):
                pred = model(xbatch.float())
                if isClassifier:
                    _, preds = torch.max(pred, 1)
                
                loss = criterion(pred, ybatch)

                # backward + optimize only if in training phase
                loss.backward()
                optimizer.step()
            
            
            # statistics
            running_loss += loss.item() * xbatch.size(0)
            if isClassifier:
                running_corrects += torch.sum(preds == ybatch.data)

            # print ("losses_iter", loss.item() * inputs.size(0))
            # print ("accuracy_iter", torch.sum(preds == labels.data).item() / float(inputs.size(0)))

            losses_iter.append(loss.item())
            if isClassifier:
                accuracy_iter.append(torch.sum(preds == ybatch.data).item() / float(xbatch.size(0)))
        
        
        train_loss = np.mean(losses_iter)
        losses_train.append(train_loss)
        if isClassifier:
            train_acc = np.mean(accuracy_iter)
            accuracy_train.append(train_acc)
            print'Training Loss: {:.4f} Acc: {:.4f}'.format(train_loss, train_acc)  
        else:
            print'Training Loss: {:.4f}'.format(train_loss)
 
        
        
        #Validation part
        model.eval()  # Set model to training mode
        
        xbatch = torch.from_numpy(X_val).to(device)
        if isClassifier:
            ybatch = torch.from_numpy(y_val).to(device).long()
        else:
            ybatch = torch.from_numpy(y_val).to(device).float()
        
        optimizer.zero_grad()
        
        pred = model(xbatch.float())
        if isClassifier:
            _, preds = torch.max(pred, 1)

        loss = criterion(pred, ybatch)
        
        val_loss = loss.item()
        scheduler.step(val_loss)
        
        if isClassifier:
            val_acc = torch.sum(preds == ybatch.data).item() / float(xbatch.size(0))
            accuracy_val.append(val_acc)
        
        losses_val.append(val_loss)
        
        
        if isClassifier:
            print'Validation Loss: {:.4f} Acc: {:.4f}'.format(val_loss, val_acc)
        else:
            print'Validation Loss: {:.4f}'.format(val_loss)

#         deep copy the model
        if isClassifier:
            if val_acc > best_acc:
                best_train_acc = train_acc
                best_acc = val_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                path = name + "_best_model.pt"
                torch.save(best_model_wts, "classifier_files/" + path)
        else:
            if val_loss < best_loss:
                best_train_loss = train_loss
                best_loss = val_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                path = name + "_best_model.pt"
                torch.save(best_model_wts, "classifier_files/" + path)

        print ""


    time_elapsed = time.time() - since
    print'Training complete in {:.0f}s'.format(time_elapsed)
    if isClassifier:
        print'Best Train Acc: {:4f}'.format(best_train_acc)
        print'Best Val Acc: {:4f}'.format(best_acc)
    else:
        print'Best Train Loss: {:4f}'.format(best_train_loss)
        print'Best Val Loss: {:4f}'.format(best_loss)

    
#     stats_columns = ['Layers', 'Epochs', 'BatchSize', 'LearningRate', 'Optimizer', 'Scheduler', 'TrainAcc', 'ValAcc']
#     layers = [module for module in model.modules() if type(module) != nn.Sequential]
    #write_results([layers, num_epochs, minibatch_size, learning_rate, optimizer.state_dict, scheduler.state_dict, best_train_acc, best_val_acc])

    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model

## MP Classifier Training

In [56]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.drop_layer = nn.Dropout(p=0.0)
        self.fc1 = nn.Linear(11, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.drop_layer(x)
        x = F.relu(self.fc2(x))
        x = self.drop_layer(x)
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

n_epochs = 30
minibatch_size = 16
learning_rate = 0.001
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model = Net()
model = model.to(device)
ce_criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# scheduler = lr_scheduler.CosineAnnealingLR(optimizer, X_train.shape[0], eta_min=learning_rate)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
mp_model = train_model(True, X1_train, y1_train, X1_val, y1_val, model, ce_criterion, optimizer, scheduler, minibatch_size, "mp_classifier", num_epochs=n_epochs)

Epoch 0/29
----------
Training Loss: 1.3408 Acc: 0.5388
Validation Loss: 1.1678 Acc: 0.5841

Epoch 1/29
----------
Training Loss: 1.1129 Acc: 0.6079
Validation Loss: 1.0609 Acc: 0.6283

Epoch 2/29
----------
Training Loss: 1.0501 Acc: 0.6235
Validation Loss: 1.0150 Acc: 0.6239

Epoch 3/29
----------
Training Loss: 1.0405 Acc: 0.6197
Validation Loss: 0.9983 Acc: 0.6283

Epoch 4/29
----------
Training Loss: 1.0111 Acc: 0.6292
Validation Loss: 0.9911 Acc: 0.6283

Epoch 5/29
----------
Training Loss: 1.0058 Acc: 0.6320
Validation Loss: 0.9705 Acc: 0.6504

Epoch 6/29
----------
Training Loss: 0.9930 Acc: 0.6362
Validation Loss: 0.9554 Acc: 0.6504

Epoch 7/29
----------
Training Loss: 0.9744 Acc: 0.6498
Validation Loss: 0.9406 Acc: 0.6593

Epoch 8/29
----------
Training Loss: 0.9601 Acc: 0.6498
Validation Loss: 0.9319 Acc: 0.6593

Epoch 9/29
----------
Training Loss: 0.9481 Acc: 0.6511
Validation Loss: 0.9110 Acc: 0.6549

Epoch 10/29
----------
Training Loss: 0.9386 Acc: 0.6509
Validation Lo

In [59]:
print "Test Results for MP Classification"
predict(X1_test, y1_test, mp_model, ce_criterion, True)

Test Results for MP Classification
Test data, Loss: 0.931, Accuracy: 0.665


## Time Regressor Training

In [57]:
class Net_Regressor(nn.Module):
    def __init__(self):
        super(Net_Regressor, self).__init__()
        self.drop_layer = nn.Dropout(p=0.15)
        self.fc1 = nn.Linear(11, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.drop_layer(x)
        x = F.relu(self.fc2(x))
        x = self.drop_layer(x)
        x = self.fc3(x)
        return x
    
n_epochs = 30
minibatch_size = 16
learning_rate = 0.001
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model = Net_Regressor()
model = model.to(device)
mse_criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# scheduler = lr_scheduler.CosineAnnealingLR(optimizer, X_train.shape[0], eta_min=learning_rate)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
time_model = train_model(False, X2_train, y2_train, X2_val, y2_val, model, mse_criterion, optimizer, scheduler, minibatch_size, 'time_regressor', num_epochs=n_epochs)

Epoch 0/29
----------
Training Loss: 2.4602
Validation Loss: 1.6790

Epoch 1/29
----------
Training Loss: 1.0479
Validation Loss: 0.7556

Epoch 2/29
----------
Training Loss: 0.8396
Validation Loss: 0.6843

Epoch 3/29
----------
Training Loss: 0.7531
Validation Loss: 0.6565

Epoch 4/29
----------
Training Loss: 0.7147
Validation Loss: 0.6238

Epoch 5/29
----------
Training Loss: 0.7215
Validation Loss: 0.6196

Epoch 6/29
----------
Training Loss: 0.7047
Validation Loss: 0.6188

Epoch 7/29
----------
Training Loss: 0.7039
Validation Loss: 0.6145

Epoch 8/29
----------
Training Loss: 0.7150
Validation Loss: 0.6058

Epoch 9/29
----------
Training Loss: 0.7060
Validation Loss: 0.6007

Epoch 10/29
----------
Training Loss: 0.6747
Validation Loss: 0.6010

Epoch 11/29
----------
Training Loss: 0.6855
Validation Loss: 0.5997

Epoch 12/29
----------
Training Loss: 0.6830
Validation Loss: 0.5945

Epoch 13/29
----------
Training Loss: 0.6543
Validation Loss: 0.5791

Epoch 14/29
----------
Trainin

In [58]:
print "Test Results for Time Regression"
predict(X2_test, y2_test, time_model, mse_criterion, False)

Test Results for Time Regression
Test data, Loss: 0.579

Actual - Predicted
2.54 - 1.64
1.16 - 1.5
2.61 - 1.62
1.48 - 1.65
2.32 - 1.83
3.29 - 1.65
2.88 - 1.43
1.88 - 1.48
0.292 - 1.52
2.51 - 1.64
2.17 - 1.86
2.64 - 1.76
1.27 - 1.4
2.6 - 1.32
1.64 - 1.53
1.36 - 1.53
1.7 - 1.68
0.305 - 1.52
1.48 - 1.38
1.8 - 1.43
1.7 - 1.58
1.63 - 1.52
1.84 - 1.67
1.47 - 1.56
2.1 - 1.88
1.25 - 1.55
1.42 - 1.43
0.456 - 1.62
0.934 - 1.38
1.35 - 1.51
3.71 - 1.63
1.52 - 1.44
1.05 - 1.48
0.739 - 1.74
1.33 - 1.54
1.82 - 1.69
2.47 - 1.72
1.05 - 1.49
1.24 - 1.53
1.08 - 1.4
1.34 - 1.62
2.01 - 1.52
2.11 - 1.93
1.22 - 1.59
1.93 - 1.57
1.68 - 1.43
1.41 - 1.37
1.76 - 1.57
0.645 - 1.49
3.01 - 1.65
2.87 - 1.79
2.34 - 1.79
1.09 - 1.67
1.85 - 1.65
2.55 - 1.75
2.42 - 1.72
2.11 - 1.58
1.49 - 1.4
0.968 - 1.57
2.33 - 1.7
1.25 - 1.45
0.753 - 1.36
1.28 - 1.67
1.41 - 1.49
2.96 - 1.81
0.263 - 1.56
2.35 - 1.63
0.994 - 1.65
0.294 - 1.51
2.03 - 1.64
1.49 - 1.59
1.33 - 1.56
0.943 - 1.48
0.812 - 1.8
2.58 - 1.73
